# Econometrics 2 - Assignment 2
## Imports

In [105]:
!pip install numpy pandas scipy statsmodels

In [106]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import statsmodels.api as stm
from statsmodels.sandbox.regression.gmm import IV2SLS

## Import data

In [107]:
df = pd.read_excel("IV_01.xlsx")
df

,Y,X1,Z1
0,-0.408071,-3.578101,0
1,5.011391,3.707990,0
2,6.471941,5.544009,0
3,-0.667804,-1.256486,0
4,2.464373,0.137812,0
...,...,...,...
1495,0.281510,3.211403,1
1496,0.599403,0.370344,1
1497,-0.128823,-1.363617,1
1498,5.647563,7.930720,1


## a
Geen code.

## b

In [108]:
x_reg = stm.OLS(df["X1"], stm.add_constant(df["Z1"])).fit()
x_reg.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.102    
Dependent Variable: X1               AIC:                7720.3166
Date:               2022-05-18 15:35 BIC:                7730.9431
No. Observations:   1500             Log-Likelihood:     -3858.2  
Df Model:           1                F-statistic:        170.7    
Df Residuals:       1498             Prob (F-statistic): 5.07e-37 
R-squared:          0.102            Scale:              10.051   
--------------------------------------------------------------------
            Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const      -1.0208     0.1158   -8.8181   0.0000   -1.2479   -0.7937
Z1          2.1389     0.1637   13.0650   0.0000    1.8178    2.4601
------------------------------------------------------------------
Omnibus:               1.076        Durbin-Watson:           2.089
Prob(Omnibus):         0.584        Jarque-Bera (JB):        1.067
Skew:                  0.065        Prob(JB):                0.587
Kurtosis:              2.995        Condition No.:           3    
==================================================================

"""

Thus we get a t-stat of 13.065 > 3.16, thus is relevant.

## c

In [109]:
x_hat = x_reg.predict(stm.add_constant(df["Z1"]))
x_hat

0      -1.020819
1      -1.020819
2      -1.020819
3      -1.020819
4      -1.020819
          ...   
1495    1.118114
1496    1.118114
1497    1.118114
1498    1.118114
1499    1.118114
Length: 1500, dtype: float64

In [110]:
v_hat = df["X1"] - x_hat
df_v_hat = df.copy()
df_v_hat.loc[:, "V_hat"] = v_hat
v_hat

0      -2.557281
1       4.728809
2       6.564828
3      -0.235666
4       1.158631
          ...   
1495    2.093289
1496   -0.747770
1497   -2.481731
1498    6.812606
1499    0.744983
Length: 1500, dtype: float64

In [111]:
y_reg = stm.OLS(df["Y"], stm.add_constant(df_v_hat[["X1", "V_hat"]])).fit()
y_reg.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.832    
Dependent Variable: Y                AIC:                5225.2405
Date:               2022-05-18 15:35 BIC:                5241.1801
No. Observations:   1500             Log-Likelihood:     -2609.6  
Df Model:           2                F-statistic:        3705.    
Df Residuals:       1497             Prob (F-statistic): 0.00     
R-squared:          0.832            Scale:              1.9034   
--------------------------------------------------------------------
           Coef.    Std.Err.      t       P>|t|     [0.025    0.975]
--------------------------------------------------------------------
const      0.1142     0.0357     3.2024   0.0014    0.0442    0.1841
X1        -0.9294     0.0333   -27.9028   0.0000   -0.9947   -0.8640
V_hat      1.8450     0.0352    52.4824   0.0000    1.7760    1.9139
------------------------------------------------------------------
Omnibus:              4.686         Durbin-Watson:           1.988
Prob(Omnibus):        0.096         Jarque-Bera (JB):        3.955
Skew:                 -0.037        Prob(JB):                0.138
Kurtosis:             2.759         Condition No.:           6    
==================================================================

"""

Thus we get t-stat = 52.48 > $\sqrt{10}$, thus we have endogeneity.

## d
Geen code.

## e
### IV:

In [112]:
cov_z1_y = df["Z1"].cov(df["Y"])
cov_z1_x1 = df["Z1"].cov(df["X1"])
beta_1_IV = cov_z1_y/cov_z1_x1
f"Beta1,IV = {beta_1_IV}"

'Beta1,IV = -0.9293842177390383'

Je kan ook zo berekenen.

In [113]:
f'Beta_1,IV = {((df["Z1"] - df["Z1"].mean()) * (df["Y"] - df["Y"].mean())).sum()/((df["Z1"] - df["Z1"].mean()) * (df["X1"] - df["X1"].mean())).sum()}'

'Beta_1,IV = -0.9293842177390383'

### 2SLS:

In [114]:
IV2SLS(df["Y"], stm.add_constant(df["X1"]), stm.add_constant(df["Z1"])).fit().summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: IV2SLS
=================================================================
Model:              IV2SLS           R-squared:          -2.191  
Dependent Variable: Y                Adj. R-squared:     -2.193  
Date:               2022-05-18 15:35 F-statistic:        41.03   
No. Observations:   1500             Prob (F-statistic): 2.00e-10
Df Model:           1                Scale:              36.115  
Df Residuals:       1498                                         
-------------------------------------------------------------------
           Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
-------------------------------------------------------------------
const      0.1142     0.1553    0.7352   0.4623   -0.1905    0.4189
X1        -0.9294     0.1451   -6.4056   0.0000   -1.2140   -0.6448
-----------------------------------------------------------------
Omnibus:              1.603        Durbin-Watson:           2.068
Prob(Omnibus):        0.449        Jarque-Bera (JB):        1.627
Skew:                 0.079        Prob(JB):                0.443
Kurtosis:             2.970        Condition No.:           3    
=================================================================

"""

In [115]:
stm.OLS(df["Y"], stm.add_constant(x_hat)).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     143.5
Date:                Wed, 18 May 2022   Prob (F-statistic):           1.22e-31
Time:                        15:35:56   Log-Likelihood:                -3878.6
No. Observations:                1500   AIC:                             7761.
Df Residuals:                    1498   BIC:                             7772.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1142      0.083      1.375      0.169      -0.049       0.277
0             -0.9294      0.078    -11.978      0.000      -1.082      -0.777
==============================================================================
Omnibus:                        1.558   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.459   Jarque-Bera (JB):                1.614
Skew:                           0.075   Prob(JB):                        0.446
Kurtosis:                       2.941   Cond. No.                         1.09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Beide geeft dezelfde resultaten, maar is net een andere manier van berekenen (al komt op hetzelfde neer).

## f

### OLS:

In [116]:
stm.OLS(df["Y"], stm.add_constant(df["X1"])).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.523
Model:                            OLS   Adj. R-squared:                  0.522
Method:                 Least Squares   F-statistic:                     1640.
Date:                Wed, 18 May 2022   Prob (F-statistic):          7.46e-243
Time:                        15:35:56   Log-Likelihood:                -3392.5
No. Observations:                1500   AIC:                             6789.
Df Residuals:                    1498   BIC:                             6800.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0336      0.060      0.560      0.575      -0.084       0.151
X1             0.7269      0.018     40.503      0.000       0.692       0.762
==============================================================================
Omnibus:                       85.504   Durbin-Watson:                   0.831
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               33.866
Skew:                          -0.082   Prob(JB):                     4.43e-08
Kurtosis:                       2.282   Cond. No.                         3.34
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 2SLS:

In [117]:
IV2SLS(df["Y"], stm.add_constant(df["X1"]), stm.add_constant(df["Z1"])).fit().summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: IV2SLS
=================================================================
Model:              IV2SLS           R-squared:          -2.191  
Dependent Variable: Y                Adj. R-squared:     -2.193  
Date:               2022-05-18 15:35 F-statistic:        41.03   
No. Observations:   1500             Prob (F-statistic): 2.00e-10
Df Model:           1                Scale:              36.115  
Df Residuals:       1498                                         
-------------------------------------------------------------------
           Coef.    Std.Err.      t      P>|t|     [0.025    0.975]
-------------------------------------------------------------------
const      0.1142     0.1553    0.7352   0.4623   -0.1905    0.4189
X1        -0.9294     0.1451   -6.4056   0.0000   -1.2140   -0.6448
-----------------------------------------------------------------
Omnibus:              1.603        Durbin-Watson:           2.068
Prob(Omnibus):        0.449        Jarque-Bera (JB):        1.627
Skew:                 0.079        Prob(JB):                0.443
Kurtosis:             2.970        Condition No.:           3    
=================================================================

"""

The confidence intervals do not overlap, which is in line with our Durbin-Hausmann test.

## g

In [118]:
!pip install linearmodels

In [119]:
from linearmodels.iv.model import IV2SLS as IV2SLS_

In [120]:
IV2SLS_model = IV2SLS_(df["Y"], np.ones(1500), df["X1"], df["Z1"]).fit(cov_type="unadjusted")
IV2SLS_model.sargan

Invalid test statistic
Test requires more instruments than endogenous variables.
Sargan's test of overidentification
InvalidTestStatistic, id: 0x7fc5b1ece580

Ik heb oprecht geen idee, maar hij zegt dat het alleen kan als je meerdere instruments hebt. Hier onder heb ik gedaan zoals staat in de slides handmatig, maar dat geeft een extreem lage waarde.

In [121]:
model = stm.OLS(df["Y"], stm.add_constant(x_hat)).fit()
u_hat = (df["Y"] - model.predict(stm.add_constant(x_hat)))
stm.OLS(u_hat, stm.add_constant(df["Z1"])).fit().rsquared * 1500

1.6653345369377348e-13

## h

In [122]:
y_star = df["Y"] - 0.1 * df["Z1"]
IV2SLS_(y_star, np.ones(1500), df["X1"], df["Z1"]).fit(cov_type="unadjusted")

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:              dependent   R-squared:                     -2.3164
Estimator:                    IV-2SLS   Adj. R-squared:                -2.3186
No. Observations:                1500   F-statistic:                    43.223
Date:                Wed, May 18 2022   P-value (F-stat)                0.0000
Time:                        15:35:59   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
exog           0.0665     0.1590     0.4182     0.6758     -0.2451      0.3780
X1            -0.9761     0.1485    -6.5744     0.0000     -1.2671     -0.6851
==============================================================================

Endogenous: X1
Instruments: Z1
Unadjusted Covariance (Homoskedastic)
Debiased: False
IVResults, id: 0x7fc5b1ecebe0

## i

In [123]:
df_ml = pd.read_excel("OLS_and_ML_01.xlsx")
df_ml["X1^2"] = df_ml["X1"] ** 2
df_ml

,Y,X1,X1^2
0,33.212895,8.565474,73.367351
1,4.517619,4.533346,20.551227
2,2.753637,6.018066,36.217116
3,38.236895,9.126797,83.298422
4,36.177911,8.800101,77.441781
...,...,...,...
1495,25.483493,7.848047,61.591835
1496,25.333941,8.098631,65.587824
1497,15.565423,6.943013,48.205427
1498,-0.667540,2.397400,5.747527


In [146]:
from statsmodels.stats.diagnostic import linear_reset

ml_model = stm.OLS(df_ml["Y"], stm.add_constant(df_ml["X1"])).fit()
display(ml_model.summary())
display(linear_reset(ml_model, use_f=True, cov_type="HC0").summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.683
Model:                            OLS   Adj. R-squared:                  0.683
Method:                 Least Squares   F-statistic:                     3224.
Date:                Wed, 18 May 2022   Prob (F-statistic):               0.00
Time:                        15:50:57   Log-Likelihood:                -5175.7
No. Observations:                1500   AIC:                         1.036e+04
Df Residuals:                    1498   BIC:                         1.037e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.4779      0.399    -13.716      0.000      -6.261      -4.694
X1             3.9215      0.069     56.780      0.000       3.786       4.057
==============================================================================
Omnibus:                      121.233   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              100.810
Skew:                           0.553   Prob(JB):                     1.29e-22
Kurtosis:                       2.375   Cond. No.                         12.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

/Users/micha/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


'<F test: F=array([[7182.19437767]]), p=0.0, df_denom=1.5e+03, df_num=2>'

Thus accepts $H_0$ (linear).

## j

In [147]:
squared_model = stm.OLS(df_ml["Y"], stm.add_constant(df_ml[["X1", "X1^2"]])).fit()
display(squared_model.summary())
display(linear_reset(squared_model, use_f=True, cov_type="HC0").summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                 2.254e+04
Date:                Wed, 18 May 2022   Prob (F-statistic):               0.00
Time:                        15:52:44   Log-Likelihood:                -3458.7
No. Observations:                1500   AIC:                             6923.
Df Residuals:                    1497   BIC:                             6939.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.0997      0.192     57.806      0.000      10.723      11.476
X1            -6.0241      0.089    -67.632      0.000      -6.199      -5.849
X1^2           1.0003      0.009    115.226      0.000       0.983       1.017
==============================================================================
Omnibus:                       31.346   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.622
Skew:                           0.100   Prob(JB):                     1.13e-13
Kurtosis:                       3.956   Cond. No.                         148.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

/Users/micha/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


'<F test: F=array([[0.35779667]]), p=0.6992750915944113, df_denom=1.5e+03, df_num=2>'

## k

In [126]:
sts.jarque_bera(squared_model.resid)

Jarque_beraResult(statistic=59.62159902846151, pvalue=1.1302070390684094e-13)

Thus not normally distributed.

## l

In [127]:
from statsmodels.miscmodels.tmodel import TLinearModel
ml_t_model = TLinearModel(df_ml["Y"], stm.add_constant(df_ml[["X1", "X1^2"]])).fit(maxiter=1000000)
ml_t_model.summary()

Optimization terminated successfully.
         Current function value: 2.295276
         Iterations: 229
         Function evaluations: 375


/Users/micha/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:2653: UserWarning: df_model + k_constant differs from nparams
  warnings.warn("df_model + k_constant differs from nparams")
/Users/micha/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:2655: UserWarning: df_resid differs from nobs - nparams
  warnings.warn("df_resid differs from nobs - nparams")


<class 'statsmodels.iolib.summary.Summary'>
"""
                             TLinearModel Results                             
==============================================================================
Dep. Variable:                      Y   Log-Likelihood:                -3442.9
Model:                   TLinearModel   AIC:                             6892.
Method:            Maximum Likelihood   BIC:                             6908.
Date:                Wed, 18 May 2022                                         
Time:                        15:36:01                                         
No. Observations:                1500                                         
Df Residuals:                    1497                                         
Df Model:                           2                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.0907      0.184     60.158      0.000      10.729      11.452
X1            -6.0320      0.086    -70.499      0.000      -6.200      -5.864
X1^2           1.0014      0.008    120.226      0.000       0.985       1.018
df             9.0162      1.939      4.650      0.000       5.216      12.816
scale          2.1438      0.065     33.151      0.000       2.017       2.271
==============================================================================
"""

## m

In [128]:
u_hat_2 = df_ml["Y"] - ml_t_model.predict(stm.add_constant(df_ml[["X1", "X1^2"]]))

sts.jarque_bera(sts.norm.ppf(sts.t.cdf(u_hat_2, df=ml_t_model.params[3], scale=ml_t_model.params[4])))

Jarque_beraResult(statistic=0.6157706564729434, pvalue=0.7349995968767395)

## n
Geen code.

## o
Geen code.

## p

In [129]:
ml_t_model_small = TLinearModel(df_ml["Y"], stm.add_constant(df_ml["X1"])).fit(maxiter=1000000)
display(ml_t_model_small.summary())
u_hat_3 = df_ml["Y"] - ml_t_model_small.predict(stm.add_constant(df_ml["X1"]))
sts.jarque_bera(sts.norm.ppf(sts.t.cdf(u_hat_3, df=ml_t_model_small.params[2], scale=ml_t_model_small.params[3])))

Optimization terminated successfully.
         Current function value: 3.450474
         Iterations: 412
         Function evaluations: 783


/Users/micha/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:2653: UserWarning: df_model + k_constant differs from nparams
  warnings.warn("df_model + k_constant differs from nparams")
/Users/micha/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:2655: UserWarning: df_resid differs from nobs - nparams
  warnings.warn("df_resid differs from nobs - nparams")


<class 'statsmodels.iolib.summary.Summary'>
"""
                             TLinearModel Results                             
==============================================================================
Dep. Variable:                      Y   Log-Likelihood:                -5175.7
Model:                   TLinearModel   AIC:                         1.036e+04
Method:            Maximum Likelihood   BIC:                         1.037e+04
Date:                Wed, 18 May 2022                                         
Time:                        15:36:01                                         
No. Observations:                1500                                         
Df Residuals:                    1498                                         
Df Model:                           1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.4770      0.399    -13.722      0.000      -6.259      -4.695
X1             3.9214      0.069     56.813      0.000       3.786       4.057
df          7.342e+06   3.79e+05     19.367      0.000     6.6e+06    8.08e+06
scale          7.6262      0.139     54.765      0.000       7.353       7.899
==============================================================================
"""

Jarque_beraResult(statistic=100.80896045829437, pvalue=0.0)